<a href="https://colab.research.google.com/github/Charity-Githogora/sms-spam-classifier/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install and import required libraries

This cell installs TensorFlow (latest nightly version) and `tensorflow-datasets`, then imports necessary libraries like TensorFlow, pandas, NumPy, and matplotlib for building and training the model.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

### Download dataset

This cell downloads the SMS Spam Collection training and validation datasets (in TSV format) using `wget`, and defines file paths for loading them.


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

### Load and preprocess data

We load the training and test datasets into pandas DataFrames, map labels (`ham`/`spam`) to binary values (0/1), tokenize the messages, and pad them to a uniform length for neural network input.


In [ ]:
# Load data
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=["label", "message"])
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=["label", "message"])

# Convert labels to binary: ham = 0, spam = 1
train_data['label'] = train_data['label'].map({'ham': 0, 'spam': 1})
test_data['label'] = test_data['label'].map({'ham': 0, 'spam': 1})

# Split inputs and outputs
X_train = train_data['message'].values
y_train = train_data['label'].values
X_test = test_data['message'].values
y_test = test_data['label'].values

# Tokenize the text
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences
X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=100, padding='post')
X_test_padded = tf.keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=100, padding='post')


### Build, train the model, and define prediction function

We create a Bidirectional LSTM neural network to classify SMS messages. The model is compiled, trained, and a `predict_message` function is defined to return the predicted probability and label ("ham" or "spam").


In [ ]:
# Build a better model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=1000, output_dim=32, input_length=100),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model (10–15 epochs recommended)
history = model.fit(X_train_padded, y_train, epochs=15, validation_data=(X_test_padded, y_test), verbose=2)

# Predict function
def predict_message(pred_text):
  seq = tokenizer.texts_to_sequences([pred_text])
  padded = tf.keras.preprocessing.sequence.pad_sequences(seq, maxlen=100, padding='post')
  pred = model.predict(padded)[0][0]
  label = 'spam' if pred > 0.5 else 'ham'
  return [float(pred), label]


### Test a single custom message

You can test how the model classifies any custom message by calling the `predict_message` function here.


In [ ]:
# Test a single message prediction
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

### Test model with predefined messages

This cell tests the model on a set of 7 known messages. If all are classified correctly, the challenge is marked as passed.


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
